In [ ]:
%load_ext autoreload
%autoreload 2
from airo_drake.stations.ur3e_cart import MakeUR3eCartStation, RunAndPublishSimulation
from airo_drake.planners.planners_base import DualArmPlannerBase

from pydrake.geometry import StartMeshcat

In [ ]:
meshcat = StartMeshcat()

In [ ]:
additional_directives = """- add_model:
        name: brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
"""
station = MakeUR3eCartStation(additional_directives)
plant = station.GetSubsystemByName("plant")

In [ ]:
from pydrake.all import PiecewisePose, RigidTransform, RotationMatrix
from airo_drake.geometry import top_down_orientation
import numpy as np

class GraspPlanner(DualArmPlannerBase):
    def __init__(self, plant, meshcat):
        super().__init__(plant, meshcat)   
        self.planned = False     

    def Plan(self, context, state):
        if self.planned:
            return
        grasp_pose = self.inital_right_tcp
        pregrasp_pose = self.inital_right_tcp

        global_y_axis = np.array([0, 1, 0])
        grasp_orientation = RotationMatrix(top_down_orientation(gripper_open_direction=global_y_axis))
        grasp_position= np.array([0, 0, 0.1])
        grasp_pose = RigidTransform(grasp_orientation, grasp_position)

        self.right_tcp_keyposes = {"initial": self.inital_right_tcp, "pregrasp": pregrasp_pose, "grasp": grasp_pose}
        times = {"initial": 0.0, "pregrasp": 2.0, "grasp": 2.5}

        def MakeLinearTrajectory(keyposes_dict, times_dict):
            keyposes = list(keyposes_dict.values())
            times = list(times_dict.values())
            pose_trajectory = PiecewisePose.MakeLinear(times, keyposes)
            return pose_trajectory

        self.right_tcp_trajectory = MakeLinearTrajectory(self.right_tcp_keyposes, times)
        self.UpdatePlanVisualization()
        self.planned = True

planner = GraspPlanner(plant, meshcat)

In [ ]:
RunAndPublishSimulation(station, planner, meshcat)